In [1]:
import pandas as pd
import yfinance as yfin
from pandas_datareader import data as pdr

bradesco_ticker = "BBD"
banco_brasil_ticker = "BDORY"

start_date = "2020-04-01"
end_date = "2023-07-01"

yfin.pdr_override()

Bradesco = pdr.get_data_yahoo(bradesco_ticker, start=start_date, end=end_date)
Banco_Brasil = pdr.get_data_yahoo(banco_brasil_ticker, start=start_date, end=end_date)

multi_index_df = pd.concat([Bradesco, Banco_Brasil], axis=1, keys=[bradesco_ticker, banco_brasil_ticker])

multi_index_df.columns.names = ['Stock Ticker', 'Stock Info']


yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
Bradesco

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-04-01,3.78,3.84,3.65,3.78,2.915777,14219900
2020-04-02,3.79,3.87,3.70,3.79,2.923491,12103900
2020-04-03,3.64,3.67,3.41,3.55,2.738363,18305200
2020-04-06,3.95,4.06,3.88,3.96,3.054625,19423100
2020-04-07,4.35,4.36,4.11,4.12,3.178044,22619700
...,...,...,...,...,...,...
2023-06-26,3.51,3.54,3.43,3.46,3.067896,22418600
2023-06-27,3.41,3.43,3.35,3.42,3.068794,27701500
2023-06-28,3.35,3.37,3.31,3.32,2.979063,21125000


In [5]:
Bradesco.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 818 entries, 2020-04-01 to 2023-06-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       818 non-null    float64
 1   High       818 non-null    float64
 2   Low        818 non-null    float64
 3   Close      818 non-null    float64
 4   Adj Close  818 non-null    float64
 5   Volume     818 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 44.7 KB


In [6]:
Banco_Brasil

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-04-01,2.530,2.585,2.450,2.460,1.676703,779200
2020-04-02,2.545,2.600,2.485,2.590,1.765309,505600
2020-04-03,2.485,2.485,2.325,2.380,1.622176,915600
2020-04-06,2.610,2.680,2.490,2.605,1.775533,1132600
2020-04-07,2.890,2.895,2.800,2.845,1.939113,579000
...,...,...,...,...,...,...
2023-06-26,5.245,5.385,5.240,5.365,4.784481,485800
2023-06-27,5.215,5.375,5.170,5.180,4.619500,102400
2023-06-28,5.250,5.250,5.060,5.105,4.552615,125000


In [7]:
Banco_Brasil.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 818 entries, 2020-04-01 to 2023-06-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       818 non-null    float64
 1   High       818 non-null    float64
 2   Low        818 non-null    float64
 3   Close      818 non-null    float64
 4   Adj Close  818 non-null    float64
 5   Volume     818 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 44.7 KB


### Pre Processamento

- Preenchendo datas faltantes (nao-comerciais) e tornando-as o novo index de cada DataFrame
- Para cada coluna do DataFrame, devemos completar os valor NaN (vazios) com o valor do dia anterior, usando o metodo ffill()

In [11]:
def fill_missing_dates(dataframe):
    start = dataframe.index[0].date()
    end = dataframe.index[len(dataframe) - 1].date()

    new_dates = pd.date_range(start=start, end=end, freq="D")
    dataframe = dataframe.reindex(new_dates)
    dataframe = dataframe.rename_axis("Date")

    for column in ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']:
        dataframe[column] = dataframe[column].ffill()

    return dataframe

Bradesco = fill_missing_dates(Bradesco)
Banco_Brasil = fill_missing_dates(Banco_Brasil)


In [12]:
Bradesco

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-04-01,3.78,3.84,3.65,3.78,2.915777,14219900.0
2020-04-02,3.79,3.87,3.70,3.79,2.923491,12103900.0
2020-04-03,3.64,3.67,3.41,3.55,2.738363,18305200.0
2020-04-04,3.64,3.67,3.41,3.55,2.738363,18305200.0
2020-04-05,3.64,3.67,3.41,3.55,2.738363,18305200.0
...,...,...,...,...,...,...
2023-06-26,3.51,3.54,3.43,3.46,3.067896,22418600.0
2023-06-27,3.41,3.43,3.35,3.42,3.068794,27701500.0
2023-06-28,3.35,3.37,3.31,3.32,2.979063,21125000.0


In [13]:
Banco_Brasil

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-04-01,2.530,2.585,2.450,2.460,1.676703,779200.0
2020-04-02,2.545,2.600,2.485,2.590,1.765309,505600.0
2020-04-03,2.485,2.485,2.325,2.380,1.622176,915600.0
2020-04-04,2.485,2.485,2.325,2.380,1.622176,915600.0
2020-04-05,2.485,2.485,2.325,2.380,1.622176,915600.0
...,...,...,...,...,...,...
2023-06-26,5.245,5.385,5.240,5.365,4.784481,485800.0
2023-06-27,5.215,5.375,5.170,5.180,4.619500,102400.0
2023-06-28,5.250,5.250,5.060,5.105,4.552615,125000.0


### Visualizando os precos de fechamento dos ativos

In [14]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

In [15]:
px.line(multi_index_df.xs(key="Close", axis=1, level="Stock Info"))

### Treinando o modelo ARIMA

In [17]:
df = Bradesco['Close']

In [ ]:
# utiliza apenas a coluna 'Close' e o indice 
train = df[:'2023-06-25'] # grande parte da amostra separada para treino
test = df['2023-06-26':]

px.line(pd.concat([train['2023-01-01':], test], axis=1, keys=['Treino', 'Teste']))

In [19]:
from pmdarima import auto_arima

m_sarima = auto_arima(train,
                      random_state=42,
                      start_p=0,
                      start_q=0,
                      max_p=100,
                      max_q=100,
                      d=6, D=None,
                      #m=0,
                      seasonal=False,
                      start_Q=1, start_P=1, max_P=10, max_Q=10,
                      trace=True,
                      error_action='ignore', suppress_warnings=True,
                      stepwise=False
                      )

 ARIMA(0,6,0)(0,0,0)[0]             : AIC=4300.644, Time=0.05 sec
 ARIMA(0,6,1)(0,0,0)[0]             : AIC=inf, Time=0.36 sec
 ARIMA(0,6,2)(0,0,0)[0]             : AIC=inf, Time=0.42 sec
 ARIMA(0,6,3)(0,0,0)[0]             : AIC=inf, Time=1.19 sec
 ARIMA(0,6,4)(0,0,0)[0]             : AIC=inf, Time=1.20 sec
 ARIMA(0,6,5)(0,0,0)[0]             : AIC=-1529.383, Time=0.82 sec
 ARIMA(1,6,0)(0,0,0)[0]             : AIC=2861.018, Time=0.05 sec
 ARIMA(1,6,1)(0,0,0)[0]             : AIC=inf, Time=0.70 sec
 ARIMA(1,6,2)(0,0,0)[0]             : AIC=inf, Time=0.76 sec
 ARIMA(1,6,3)(0,0,0)[0]             : AIC=inf, Time=1.76 sec
 ARIMA(1,6,4)(0,0,0)[0]             : AIC=inf, Time=2.43 sec
 ARIMA(2,6,0)(0,0,0)[0]             : AIC=2061.882, Time=0.08 sec
 ARIMA(2,6,1)(0,0,0)[0]             : AIC=inf, Time=0.61 sec
 ARIMA(2,6,2)(0,0,0)[0]             : AIC=inf, Time=0.75 sec
 ARIMA(2,6,3)(0,0,0)[0]             : AIC=inf, Time=2.05 sec
 ARIMA(3,6,0)(0,0,0)[0]             : AIC=1522.225, Time=0.10 se

In [21]:
m_sarima.fit(train)

prev_sarima, conf = m_sarima.predict(n_periods=len(test), return_conf_int=True)

px.line(pd.concat([train['2023-04-15':], test, prev_sarima], axis=1, keys=['Treino','Teste','Previsao']))

In [22]:
import numpy as np

def rmse(y, y_hat):
    return np.sqrt(np.abs((y - y_hat)/y)*100)

def mape(y, y_hat):
    return np.mean(np.abs((y - y_hat)/y)*100)

print("RMSE Error: "+str(rmse(test.values, prev_sarima.values)))
print("MAPE Error: "+str(mape(test.values, prev_sarima.values))+" %")

RMSE Error: [0.78992033 1.14981435 0.77059051 1.9259582  3.11856287]
MAPE Error: 3.1949212584298206 %
